In [1]:

! pip install python-docx
! pip install spacy_annotator

In [1]:

from docx import Document
from os import walk
from tqdm.notebook import tqdm
import spacy_annotator
import pandas as pd
import spacy


In [3]:

def annotate_by_sentence2(courts, nlp):
    annotated_data = []

    annotated_court = []
    # Get all individual sentences (using spaCy tokenizer)
    sentences = [str(x) for x in nlp(courts).sents]

    # For every sentence
    for sentence in sentences:
      annotated_sentence = []
      for token in nlp(sentence):
        annotated_sentence.append((token, token.ent_iob_, token.ent_type_))

      # Add sentence to court list
      annotated_court.append(annotated_sentence)

    # Add court to all data
    annotated_data.append(annotated_court)
    return annotated_data

In [5]:
data1 = pd.read_csv("../Files/001-57427.docx")
data2 = pd.read_csv("../Files/001-57416.docx")

print(data1.head(1))
print(data2.head(2))

#nlp = spacy.load("en_core_web_sm")

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 14-15: invalid continuation byte

In [36]:
def get_text(fn, folder_path):
  fref = open(folder_path + fn, "rb")
  doc = Document(folder_path + fn)
  text = []
  for para in doc.paragraphs:
    text.append(para.text)
    #text += para.text + "\n"

  return '\n'.join(text)

def load_annotated(folder_path):
  counter = 0
  break_counter = 0
  texts = []
  fns = next(walk(folder_path), (None, None, []))[2]  # [] if no file
  for fn in tqdm(fns):
    try:
      #texts.append(get_text(fn, folder_path))
      csv_file = pd.read_csv(folder_path + fn)
      texts.append(csv_file)
      break_counter += 1
    except:
      counter += 1

    if break_counter > 100:
      break

  print("{} files not extracted.".format(counter))

  return texts

def clean_file(courts):
  return [court.replace('\xa0', ' ') for court in courts]

In [40]:
all_data = load_annotated('../Annotated/')
all_data = clean_file(all_data)
nlp = spacy.load("en_core_web_sm")


  0%|          | 0/624 [00:00<?, ?it/s]

0 files not extracted.


In [44]:
file_name_index = 0
folder_path = "../Annotated/"
counter = 0
break_counter = 0
texts = []
fns = next(walk(folder_path), (None, None, []))[2]  # [] if no file
for fn in tqdm(fns):
    try:
        #texts.append(get_text(fn, folder_path))
        data = pd.read_csv(folder_path + fn)

        # Spacy's annotation
        spacy_df = []
        # Iterate and label for each sentence as we go
        for index, row in data.iterrows():
            par = ''.join(row['Word']).replace("['", "").replace("']", "").replace("', '", " ")
            #print(par)

            s = annotate_by_sentence2(par, nlp)[0]


            sub_tags = []
            for r in s:
                for ind_tag in r:
                    if ind_tag[1] == "O":
                        sub_tags.append("O")
                    else:
                        sub_tags.append(ind_tag[1]+ "-" + ind_tag[2])
                #print(sub_tags)
            spacy_df.append(sub_tags)



        # Now apply the regex:
        regex_words = []
        for index, row in data.iterrows():
            row_tag = row['Tag']
            sub_spacy_row = spacy_df[index]
            #print(str(sub_spacy_row) + " " + str(row_tag))

            sub_regex = []
            # Process the tags one by one
            sub_index = 0

            for label in row_tag.replace("['", "").replace("']", "").split("', '"):

                if label == "O":    # O may not hold anything interesting
                    sub_regex.append(sub_spacy_row[sub_index])
                else:
                    sub_regex.append(label)
                sub_index += 1

            regex_words.append(sub_regex)

            #break


        df = [data['Text'], data['Word'], data['POS'], regex_words]

        df = []

        for i in range(len(regex_words)):
            df.append([data['Text'][i], data['Word'][i], data['POS'][i], regex_words[i]])



        regex_df = pd.DataFrame(data=df,columns=["Text", "Word", "POS", "Tag"])
        #print(regex_df.head())
        regex_df.to_csv("../Annotated_regex/file_" + str(file_name_index) + ".csv")


        file_name_index += 1

    except:
      counter += 1

    #if break_counter > 10:
    #    break

    print("{} files not extracted.".format(counter))


0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not

  0%|          | 0/624 [00:00<?, ?it/s]

0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not extracted.
0 files not

In [25]:
folder_path = "../Annotated_regex/"
fns = next(walk(folder_path), (None, None, []))[2]  # [] if no file
print(len(fns))

data = []

# Loop and read files
for file in tqdm(fns):
    df = pd.read_csv(folder_path + file)

    # Loop over rows and append them to the array
    for index, row in df.iterrows():
        data.append(row.values.tolist()[1:5])

    #break


624


  0%|          | 0/624 [00:00<?, ?it/s]

In [26]:
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
dataset = pd.DataFrame(data, columns=["Text", "Word", "POS", "Tag"])

In [28]:
print(dataset.head())

                         Text  \
0              FOURTH SECTION   
1          CASE OF BOROVSK v.   
2  (Application no. 24528/02)   
3                    JUDGMENT   
4                 2 June 2009   

                                               Word  POS  \
0                             ['FOURTH', 'SECTION']  NaN   
1                   ['CASE', 'OF', 'BOROVSK', 'v.']  NaN   
2  ['(', 'Application', 'no', '.', '24528/02', ')']  NaN   
3                                      ['JUDGMENT']  NaN   
4                             ['2', 'June', '2009']  NaN   

                                                 Tag  
0                             ['B-COURT', 'I-COURT']  
1                               ['O', 'O', 'O', 'O']  
2  ['O', 'O', 'B-APPLICATION', 'I-APPLICATION', '...  
3                                              ['O']  
4                          ['O', 'B-DATE', 'I-DATE']  


In [29]:
dataset.to_csv("../all_annotated_regex.csv")

In [30]:
print(len(dataset))

87633


In [1]:
import pandas as pd
# Convert to have the sentence count:

df = pd.read_csv("../all_annotated_regex.csv")


In [2]:
print(df.head())

   Unnamed: 0                        Text  \
0           0              FOURTH SECTION   
1           1          CASE OF BOROVSK v.   
2           2  (Application no. 24528/02)   
3           3                    JUDGMENT   
4           4                 2 June 2009   

                                               Word  POS  \
0                             ['FOURTH', 'SECTION']  NaN   
1                   ['CASE', 'OF', 'BOROVSK', 'v.']  NaN   
2  ['(', 'Application', 'no', '.', '24528/02', ')']  NaN   
3                                      ['JUDGMENT']  NaN   
4                             ['2', 'June', '2009']  NaN   

                                                 Tag  
0                             ['B-COURT', 'I-COURT']  
1                               ['O', 'O', 'O', 'O']  
2  ['O', 'O', 'B-APPLICATION', 'I-APPLICATION', '...  
3                                              ['O']  
4                          ['O', 'B-DATE', 'I-DATE']  


In [12]:
data = []
# Index is the Sentence count
for index, row in df.iterrows():

    words = row['Word'].replace("['", "").replace("']", "").split("', '")
    tags = row['Tag'].replace("['", "").replace("']", "").split("', '")

    for i in range(len(words)):
        if i == 0:
            data.append(["Sentence: " + str(index + 1), words[i], "NNS", tags[i]])
        else:
            data.append(['NaN', words[i], "NNS", tags[i]])

In [13]:
i = 0
for el in data:
    print(el)
    i += 1
    if i > 30:
        break

['Sentence: 1', 'FOURTH', 'NNS', 'B-COURT']
['NaN', 'SECTION', 'NNS', 'I-COURT']
['Sentence: 2', 'CASE', 'NNS', 'O']
['NaN', 'OF', 'NNS', 'O']
['NaN', 'BOROVSK', 'NNS', 'O']
['NaN', 'v.', 'NNS', 'O']
['Sentence: 3', '(', 'NNS', 'O']
['NaN', 'Application', 'NNS', 'O']
['NaN', 'no', 'NNS', 'B-APPLICATION']
['NaN', '.', 'NNS', 'I-APPLICATION']
['NaN', '24528/02', 'NNS', 'I-APPLICATION']
['NaN', ')', 'NNS', 'O']
['Sentence: 4', 'JUDGMENT', 'NNS', 'O']
['Sentence: 5', '2', 'NNS', 'O']
['NaN', 'June', 'NNS', 'B-DATE']
['NaN', '2009', 'NNS', 'I-DATE']
['Sentence: 6', 'FINAL', 'NNS', 'O']
['Sentence: 7', '02/09/2009', 'NNS', 'B-DATE']
['Sentence: 8', 'This', 'NNS', 'O']
['NaN', 'judgment', 'NNS', 'O']
['NaN', 'may', 'NNS', 'O']
['NaN', 'be', 'NNS', 'O']
['NaN', 'subject', 'NNS', 'O']
['NaN', 'to', 'NNS', 'O']
['NaN', 'editorial', 'NNS', 'O']
['NaN', 'revision', 'NNS', 'O']
['NaN', '.', 'NNS', 'O']
['Sentence: 9', 'In', 'NNS', 'O']
['NaN', 'the', 'NNS', 'O']
['NaN', 'case', 'NNS', 'O']
['NaN', 

In [14]:
sentence_df = pd.DataFrame(data, columns=["Sentence #", "Word", "POS", "Tag"])
print(sentence_df.head())
sentence_df.to_csv("../sentence_df_regex.csv")

    Sentence #     Word  POS      Tag
0  Sentence: 1   FOURTH  NNS  B-COURT
1          NaN  SECTION  NNS  I-COURT
2  Sentence: 2     CASE  NNS        O
3          NaN       OF  NNS        O
4          NaN  BOROVSK  NNS        O
